## Генерация последовательности перехода от картинки к картинке
Используются веса, созданные нейросетью *(learn.ipynb)*, обученной на разных картинках.    
Веса, обученые на одной картинке изменяются, что бы через заданное количество шагов   
соответствовать весам, обученным на другой картинке.   
На каждом шаге происходит сохранение изображения в .png файл указаного размера.

In [ ]:
import numpy
import scipy.special
import imageio
import matplotlib.pyplot
# ensure the plots are inside this notebook, not an external window
%matplotlib inline

## параметры

- **Ширина** и **высота** желаемого результата   
- **Имя папки**, где будет сохранён результат    
- **количество нейронов** в скрытом слое   

**ВНИМАНИЕ!** все обученные примеры должны быть обученны именно при таком значении скрытых нейронов.

In [ ]:
width = 200
height = 200

folder = 'result'  # имя папки, где будут сохраняться картинки

neuron_count = 150

# здесь будем формировать картинку
imout = numpy.zeros(( height, width , 3))

# папка, откуда будем брать веса. 
# Имя зависит от количества скрытых нейронов
wFolder = 'W-' + str(neuron_count) + '/'

### код нейросети

In [ ]:
# neural network class definition
class neuralNetwork:
    # initialise the neural network
    def __init__(self, neuron_count,w11,w21,w31,w12,w22,w32):
        # set number of nodes in each input, hidden, output layer
        self.inpNodes = 3
        self.hd1Nodes = neuron_count
        self.hd2Nodes = neuron_count
        self.outNodes = 3

        self.wih1 = numpy.load(wFolder + w11)
        self.wh1h2= numpy.load(wFolder + w21)
        self.wh2o = numpy.load(wFolder + w31) 
        
        self.wih1_1 = numpy.load(wFolder + w11)
        self.wh1h2_1= numpy.load(wFolder + w21)
        self.wh2o_1 = numpy.load(wFolder + w31) 
        
        self.wih1_2  = numpy.load(wFolder + w12)
        self.wh1h2_2 = numpy.load(wFolder + w22)
        self.wh2o_2  = numpy.load(wFolder + w32)  

        self.activation_function = lambda x: scipy.special.expit(x)

    def setWeight( self, currentStep, countOfStep):
        cs = countOfStep-1
        for x in range(0, self.hd1Nodes):
            for y in range(0, self.inpNodes):
                self.wih1[x,y] = self.wih1_1[x,y] + ((self.wih1_2[x,y]-self.wih1_1[x,y])/cs)*currentStep
                
        for x in range(0, self.hd2Nodes):
            for y in range(0, self.hd1Nodes):
                self.wh1h2[x,y] = self.wh1h2_1[x,y] + ((self.wh1h2_2[x,y]-self.wh1h2_1[x,y])/cs)*currentStep
                
        for x in range(0,self.outNodes):
            for y in range(0,self.hd2Nodes):
                self.wh2o[x,y] = self.wh2o_1[x,y] + ((self.wh2o_2[x,y]-self.wh2o_1[x,y])/cs)*currentStep  
        

    # query the neural network
    def query(self, inputs_list):
                
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden1_inputs = numpy.dot(self.wih1, inputs)
        # calculate the signals emerging from hidden layer
        hidden1_outputs = self.activation_function(hidden1_inputs)
        
        # calculate signals into hidden2 layer
        hidden2_inputs = numpy.dot(self.wh1h2, hidden1_outputs)
        # calculate the signals emerging from hidden layer
        hidden2_outputs = self.activation_function(hidden2_inputs)        
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.wh2o, hidden2_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs
            

#### дополнительные функции

In [ ]:
def get_inputs(x,y):
    return [y/height+0.1, x/width+0.1, 1]

def generate_image():
    for x in range(0,width):
        for y in range(0,height):
            inputs = get_inputs(x,y)
            result = nn.query(inputs)
            imout[y,x,0] = result[0]*255
            imout[y,x,1] = result[1]*255
            imout[y,x,2] = result[2]*255
            
def doit(steps,f,s):  #      
    global startFileName
    global neuron_count
    global nn
    nn = neuralNetwork( neuron_count,'w' +str(f)+ '_1.npy', 'w' +str(f)+ '_2.npy', 'w' +str(f)+'_3.npy',
                                     'w' +str(s)+ '_1.npy', 'w' +str(s)+ '_2.npy', 'w' +str(s)+'_3.npy')
    for i in range(0,steps): 
        nn.setWeight(i,steps)
        generate_image()
        imageio.imwrite(folder + '/' + str(startFileName) + '.png',  imout[:, :, :].astype(numpy.uint8))
        i += 1
        startFileName += 1         

## Финальная часть

Вначале задаём число, которое будет использоваться для имени первого файла в последовательности.  
Первым агрументом у функции **doit()** указываем количество кадров, которое будет длится преобразование.  
Затем номер первого и второго изображения  
Например:
```
startFileName = 0
doit(40,7,5)
doit(40,5,3)
```
- Файлы начнут сохраняться с 0.png
- Вначале за 40 кадров произойдёт преобразование из **7** картинки в **5**  *(0.png ... 39.png)*   
- затем за 40 кадров из **5** картинки в **3**. *(40.png ... 79.png)*

In [ ]:
startFileName = 0

print('Ждите. идёт генерация')
print(f'за результатом можно следить в папке {folder}')

doit(40,7,5)
doit(40,5,3)

print('Всё')